In [1]:
import pandas as pd
import os
from scipy import stats
from matplotlib.image import imread
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from matplotlib import rc
rc('font', family = 'AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
os.chdir('/home/kiho/다운로드/유통 & 소셜 데이터')

In [3]:
gs = pd.read_csv('gs.csv', encoding = 'CP949')
gs.head()

,reg_01.gs,date.gs,gen.gs,age.gs,cat.gs,qty.gs,reg_02.gs
0,서울특별시,20160101,F,00~19,라면,7,종로구
1,서울특별시,20160101,F,20~39,과자,655,종로구
2,서울특별시,20160101,F,20~39,라면,282,종로구
3,서울특별시,20160101,F,20~39,마스크,35,종로구
4,서울특별시,20160101,F,20~39,맥주,183,종로구


In [4]:
gs_seoul = gs[gs['reg_01.gs'] == '서울특별시']
gs_seoul = gs_seoul.rename(columns = {'date.gs': 'datetime'})

In [5]:
gs_seoul.head()

,reg_01.gs,datetime,gen.gs,age.gs,cat.gs,qty.gs,reg_02.gs
0,서울특별시,20160101,F,00~19,라면,7,종로구
1,서울특별시,20160101,F,20~39,과자,655,종로구
2,서울특별시,20160101,F,20~39,라면,282,종로구
3,서울특별시,20160101,F,20~39,마스크,35,종로구
4,서울특별시,20160101,F,20~39,맥주,183,종로구


In [6]:
age = gs_seoul.groupby(['datetime','reg_02.gs','age.gs'])['qty.gs'].sum().unstack().reset_index()
age['datetime'] = age['datetime'].astype(str)
age['key'] = age['datetime'] + age['reg_02.gs']
age.head()

age.gs,datetime,reg_02.gs,00~19,20~39,40~59,60~99,key
0,20160101,강남구,245.0,7398.0,2643.0,253.0,20160101강남구
1,20160101,강동구,49.0,2008.0,894.0,14.0,20160101강동구
2,20160101,강북구,140.0,1070.0,443.0,120.0,20160101강북구
3,20160101,강서구,70.0,2853.0,661.0,35.0,20160101강서구
4,20160101,관악구,161.0,7359.0,1177.0,119.0,20160101관악구


In [7]:
sex = gs_seoul.groupby(['datetime','reg_02.gs','gen.gs'])['gen.gs'].count().unstack().reset_index()
sex['datetime'] = sex['datetime'].astype(str)
sex['key'] = sex['datetime'] + sex['reg_02.gs']
sex.head()

gen.gs,datetime,reg_02.gs,F,M,key
0,20160101,강남구,29,26,20160101강남구
1,20160101,강동구,20,18,20160101강동구
2,20160101,강북구,19,17,20160101강북구
3,20160101,강서구,22,17,20160101강서구
4,20160101,관악구,26,25,20160101관악구


In [8]:
dt_gs = gs_seoul.groupby(['datetime','reg_02.gs','cat.gs'])['qty.gs'].sum().unstack().reset_index().fillna(0)
dt_gs['datetime'] = dt_gs['datetime'].astype(str)
dt_gs['key'] = dt_gs['datetime'] + dt_gs['reg_02.gs']
dt_gs.head()

cat.gs,datetime,reg_02.gs,과자,라면,마스크,맥주,면도기,생리대,생수,숙취해소제,스타킹,아이스크림,우산,탄산음료,key
0,20160101,강남구,4370.0,1783.0,7.0,1028.0,49.0,309.0,662.0,190.0,70.0,1607.0,0.0,464.0,20160101강남구
1,20160101,강동구,1289.0,388.0,21.0,282.0,7.0,49.0,133.0,91.0,7.0,621.0,0.0,77.0,20160101강동구
2,20160101,강북구,774.0,247.0,7.0,169.0,0.0,14.0,63.0,77.0,7.0,359.0,0.0,56.0,20160101강북구
3,20160101,강서구,1480.0,535.0,14.0,591.0,42.0,35.0,162.0,162.0,14.0,444.0,0.0,140.0,20160101강서구
4,20160101,관악구,3601.0,1692.0,0.0,1127.0,42.0,134.0,521.0,233.0,92.0,1149.0,0.0,225.0,20160101관악구


In [9]:
weather = pd.read_csv('weather.csv', encoding = 'CP949')
weather_seoul = weather[weather['region_01.wt'] == '서울특별시']
weather_seoul = weather_seoul.drop(['region_id', 'region_01.wt'], axis = 1)
weather_seoul = weather_seoul.rename(columns = {'date.wt': 'datetime'})
weather_seoul = weather_seoul.rename(columns = {'region_02.wt': 'reg_02.gs'})
weather_seoul['datetime'] = weather_seoul['datetime'].astype(str)
weather_seoul['key'] = weather_seoul['datetime'] + weather_seoul['reg_02.gs']
weather_seoul.head()

,datetime,reg_02.gs,temp_max,wind_max,temp_min,temp_avg,rhumid_avg,wa_avg,rain_sum,key
2192,20160101,종로구,4.0,3.5,-3.3,1.2,73.0,1.6,0.0,20160101종로구
2193,20160102,종로구,9.5,4.5,1.0,5.7,77.0,2.0,0.0,20160102종로구
2194,20160103,종로구,9.4,4.0,5.1,6.5,81.0,1.8,0.0,20160103종로구
2195,20160104,종로구,5.3,5.1,-2.5,2.0,54.0,3.1,0.0,20160104종로구
2196,20160105,종로구,1.5,4.6,-4.8,-2.7,39.0,2.3,0.0,20160105종로구


In [10]:
dt_01 = pd.merge(dt_gs, weather_seoul, on = 'key' )
dt_02 = pd.merge(age, sex, on = 'key' )
dt = pd.merge(dt_01, dt_02, on = 'key')

In [11]:
dt.columns

Index(['datetime_x_x', 'reg_02.gs_x_x', '과자', '라면', '마스크', '맥주', '면도기', '생리대',
       '생수', '숙취해소제', '스타킹', '아이스크림', '우산', '탄산음료', 'key', 'datetime_y_x',
       'reg_02.gs_y_x', 'temp_max', 'wind_max', 'temp_min', 'temp_avg',
       'rhumid_avg', 'wa_avg', 'rain_sum', 'datetime_x_y', 'reg_02.gs_x_y',
       '00~19', '20~39', '40~59', '60~99', 'datetime_y_y', 'reg_02.gs_y_y',
       'F', 'M'],
      dtype='object')

In [12]:
dt = dt.rename(columns = {'datetime_x_x': 'datetime'})
dt = dt.rename(columns = {'reg_02.gs_x_x': 'reg_gs'})
dt = dt.drop(['key', 'datetime_y_x','reg_02.gs_y_x','datetime_x_y','reg_02.gs_x_y',
             'datetime_y_y', 'reg_02.gs_y_y'], axis = 1)

In [13]:
dt.columns

Index(['datetime', 'reg_gs', '과자', '라면', '마스크', '맥주', '면도기', '생리대', '생수',
       '숙취해소제', '스타킹', '아이스크림', '우산', '탄산음료', 'temp_max', 'wind_max',
       'temp_min', 'temp_avg', 'rhumid_avg', 'wa_avg', 'rain_sum', '00~19',
       '20~39', '40~59', '60~99', 'F', 'M'],
      dtype='object')

In [14]:
dt.head()

,datetime,reg_gs,과자,라면,마스크,맥주,면도기,생리대,생수,숙취해소제,...,temp_avg,rhumid_avg,wa_avg,rain_sum,00~19,20~39,40~59,60~99,F,M
0,20160101,강남구,4370.0,1783.0,7.0,1028.0,49.0,309.0,662.0,190.0,...,2.1,73.0,1.2,0.0,245.0,7398.0,2643.0,253.0,29,26
1,20160101,강동구,1289.0,388.0,21.0,282.0,7.0,49.0,133.0,91.0,...,0.6,81.0,0.8,0.0,49.0,2008.0,894.0,14.0,20,18
2,20160101,강북구,774.0,247.0,7.0,169.0,0.0,14.0,63.0,77.0,...,1.7,77.0,0.8,0.0,140.0,1070.0,443.0,120.0,19,17
3,20160101,강서구,1480.0,535.0,14.0,591.0,42.0,35.0,162.0,162.0,...,3.0,NaN,1.6,0.0,70.0,2853.0,661.0,35.0,22,17
4,20160101,관악구,3601.0,1692.0,0.0,1127.0,42.0,134.0,521.0,233.0,...,1.1,NaN,1.6,0.0,161.0,7359.0,1177.0,119.0,26,25


In [15]:
dt_01 = pd.read_csv('df_predict_01.csv', parse_dates = ['datetime'])
dt_02 = pd.read_csv('df_predict_02.csv', parse_dates = ['datetime'])

In [16]:
gu = dt_01['reg_gs'].unique()
gu

array(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'], dtype=object)

In [75]:
sample = dt_01[dt_01['reg_gs'] == gu[0]]
sample = sample.drop(['datetime','reg_gs'], axis = 1)
sample = sample.fillna(0)
sample1 = sample

In [76]:
sample1.head()

,과자,라면,마스크,맥주,면도기,생리대,생수,숙취해소제,스타킹,아이스크림,우산,탄산음료,temp_max,wind_max,temp_min,temp_avg,rhumid_avg,wa_avg,rain_sum
0,4370,1783,7,1028,49,309,662,190,70,1607,0,464,5.0,2.4,-2.1,2.1,73.0,1.2,0.0
25,3933,1930,28,1262,56,246,683,148,77,1410,0,303,10.2,2.4,3.7,7.1,73.0,1.1,0.0
50,3664,1748,21,1107,28,140,592,219,77,1952,7,443,9.9,2.7,6.2,7.7,80.0,0.9,0.0
75,3799,2352,49,810,49,274,973,225,120,902,0,494,6.7,3.7,-0.7,3.4,51.0,2.4,0.0
100,3877,3044,35,691,42,274,726,289,198,1367,0,507,1.8,3.3,-2.6,-1.0,36.0,1.7,0.0


In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score

In [30]:
y_target = sample['과자']
X_data = sample.drop(['과자'],axis=1,inplace=False)

In [21]:
X_train , X_test , y_train , y_test = train_test_split(X_data , 
                                                       y_target ,
                                                       test_size=0.3, 
                                                       random_state=156)

In [22]:
# Linear Regression OLS로 학습/예측/평가 수행. 
lr = LinearRegression()
lr.fit(X_train ,y_train )
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_test, y_preds)))

MSE : 333682.805 , RMSE : 577.653
Variance score : 0.483


In [23]:
#시계열 예측을 위한 Facebook Prophet 사용하기

In [24]:
gu = dt_02['reg_gs'].unique()
gu

array(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'], dtype=object)

In [25]:
sample = dt_02[dt_02['reg_gs'] == gu[0]]
sample = sample.drop(['datetime','reg_gs'], axis = 1)
sample = sample.fillna(0)

In [26]:
y_target = sample['과자']
X_data = sample.drop(['과자'],axis=1,inplace=False)

In [27]:
X_train , X_test , y_train , y_test = train_test_split(X_data , 
                                                       y_target ,
                                                       test_size=0.3, 
                                                       random_state=156)

In [28]:
# Linear Regression OLS로 학습/예측/평가 수행. 
lr = LinearRegression()
lr.fit(X_train ,y_train )
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_test, y_preds)))

MSE : 0.000 , RMSE : 0.000
Variance score : 1.000


In [64]:
import tensorflow as tf

In [34]:
sample.head()

,과자,라면,마스크,맥주,면도기,생리대,생수,숙취해소제,스타킹,아이스크림,우산,탄산음료,temp_max,wind_max,temp_min,temp_avg,rhumid_avg,wa_avg,rain_sum
0,4370,1783,7,1028,49,309,662,190,70,1607,0,464,5.0,2.4,-2.1,2.1,73.0,1.2,0.0
25,3933,1930,28,1262,56,246,683,148,77,1410,0,303,10.2,2.4,3.7,7.1,73.0,1.1,0.0
50,3664,1748,21,1107,28,140,592,219,77,1952,7,443,9.9,2.7,6.2,7.7,80.0,0.9,0.0
75,3799,2352,49,810,49,274,973,225,120,902,0,494,6.7,3.7,-0.7,3.4,51.0,2.4,0.0
100,3877,3044,35,691,42,274,726,289,198,1367,0,507,1.8,3.3,-2.6,-1.0,36.0,1.7,0.0


In [77]:
#다변인 선형회귀
xy = np.array(sample1, dtype = np.int)

In [78]:
x_data = xy[:, 1:5]
y_data = xy[:, [0]]

In [79]:
x_data = xy[:, 1: 5]

In [80]:
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([4, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

In [67]:
hypothesis = tf.matmul(X, W) + b #가설 설정
cost = tf.reduce_mean(tf.square(hypothesis - Y)) #비용함수 설정
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000005) #0.000005는 학습률
train = optimizer.minimize(cost)
sess = tf.Session() #세션 설정
sess.run(tf.global_variables_initializer()) #글로벌변수 초기화

In [68]:
#학습을 수행
for step in range(100001):
    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 500 == 0:
        print("#", step, " 손실 비용: ", cost_)
        print("- 상품 가격: ", hypo_[0])

# 0  손실 비용:  3101041.8
- 상품 가격:  [1811.7941]
# 500  손실 비용:  nan
- 상품 가격:  [nan]
# 1000  손실 비용:  nan
- 상품 가격:  [nan]
# 1500  손실 비용:  nan
- 상품 가격:  [nan]
# 2000  손실 비용:  nan
- 상품 가격:  [nan]
# 2500  손실 비용:  nan
- 상품 가격:  [nan]
# 3000  손실 비용:  nan
- 상품 가격:  [nan]
# 3500  손실 비용:  nan
- 상품 가격:  [nan]
# 4000  손실 비용:  nan
- 상품 가격:  [nan]
# 4500  손실 비용:  nan
- 상품 가격:  [nan]
# 5000  손실 비용:  nan
- 상품 가격:  [nan]
# 5500  손실 비용:  nan
- 상품 가격:  [nan]
# 6000  손실 비용:  nan
- 상품 가격:  [nan]
# 6500  손실 비용:  nan
- 상품 가격:  [nan]
# 7000  손실 비용:  nan
- 상품 가격:  [nan]
# 7500  손실 비용:  nan
- 상품 가격:  [nan]
# 8000  손실 비용:  nan
- 상품 가격:  [nan]
# 8500  손실 비용:  nan
- 상품 가격:  [nan]
# 9000  손실 비용:  nan
- 상품 가격:  [nan]
# 9500  손실 비용:  nan
- 상품 가격:  [nan]
# 10000  손실 비용:  nan
- 상품 가격:  [nan]
# 10500  손실 비용:  nan
- 상품 가격:  [nan]
# 11000  손실 비용:  nan
- 상품 가격:  [nan]
# 11500  손실 비용:  nan
- 상품 가격:  [nan]
# 12000  손실 비용:  nan
- 상품 가격:  [nan]
# 12500  손실 비용:  nan
- 상품 가격:  [nan]
# 13000  손실 비용:  nan
- 상품 가격:  [nan]
# 13500  손실 비

In [69]:
saver = tf.train.Saver()
save_path = saver.save(sess, "./saved.cpkt")
print('학습된 모델을 저장했습니다.')

학습된 모델을 저장했습니다.


In [83]:
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([4, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

hypothesis = tf.matmul(X, W) + b #가설 설정

saver = tf.train.Saver()
model = tf.global_variables_initializer()

In [84]:
noodle = float(input('라면: '))
mask = float(input('마스크: '))
shave = float(input('면도기: '))
beer = float(input('맥주: '))

라면: 1000
마스크: 9
면도기: 20
맥주: 1000


In [85]:
with tf.Session() as sess:
    sess.run(model)
    #저장된 학습모델 불러오기
    save_path = "./saved.cpkt"
    saver.restore(sess, save_path)
    # 사용자의 입력 값을 이용해 배열을 만듭니다.
    data = ((noodle, mask, shave, beer), )
    arr = np.array(data, dtype=np.float32)
    # 예측을 수행한 뒤에 그 결과를 출력합니다.
    x_data = arr[0:4]
    dict = sess.run(hypothesis, feed_dict={X: x_data})
    print(dict[0])

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./saved.cpkt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key bias_1 not found in checkpoint
	 [[node save_1/RestoreV2 (defined at <ipython-input-83-14d92937f3b1>:8) ]]

Caused by op 'save_1/RestoreV2', defined at:
  File "/home/kiho/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kiho/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kiho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kiho/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kiho/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self